# AI-generated fake reviews detector

Online  reviews  on  digital  platforms  are  known  to  signif-icantly  influence  consumer  decisions  and  trust  in  prod-ucts  and  services. While  online  reviews  are  vital  for  consumer  decision-making, the credibility of UGC is increasingly compromised by the prevalence of fake reviews. These deceptive reviews, whether overly positive or negative, are intended to manipu-late consumer perceptions and distort the online marketplace.  Large Language Models (LLMs) are becoming indistinguishable at producing content with high accessibility and lowcost, pose a newchallenge in the proliferation of machine-generated fake reviews.

In this project, I build a fake review detector to identify AI-generated fake reviews. The details of generated dataset can be found at [Gambetti and Han (2024)](https://ojs.aaai.org/index.php/ICWSM/article/view/31437) and the link of the data is available at [here](https://zenodo.org/records/10511456)

The variables are as follows:

Variables description:

- ID: Unique identifier. It maps to each review, either generated or authentic, and to each image.
text: Review text.
- label: Binary label indicating the class (0=authentic, 1=machine-generated).
- automated_readability_index: Approximate US grade level needed to comprehend the text.
- difficult_words: Number of difficult words from Dale-Chall world list.
- flesch_reading_ease: Score on a scale from 0 to 100, with higher scores indicating easier readability.
gunning_fog: Years of formal education a person needs to understand a text easily.
- words_per_sentence: Average number of words per sentence.
- reading_time: Reading time.
- ppl: Perplexity score from zero-shot GPTNeo 125M.
- bright: Brightness. Average of V of the HSV image representation.
- sat: Saturation. Color intensity and purity of an image. Average of S of the HSV image representation.
- clar: Clarity. Well-defined objects in space. % of normalized V pixels that exceed 0.7 of HSV.
- cont: Contrast. Spread of illumination. Standard deviation of V of the HSV image representation.
- warm: Warmth. Warm colors: from red to yellow. % of H<60 or
 than 220 of HSV.
- colorf: Colorfulness. Departure from a grey-scale image.
- sd: Size difference. Difference in the number of pixels between the figure and the ground
- cd: Color difference. Difference of Euclidian distance between the figure and ground (RGB vectors).
- td: Texture difference. Absolute difference between the foreground and background edge density.
- diag_dom: Diagonal dominance. Manhattan distance between salient region and each diagonal.
- rot: Rule of thirds. Minimum distance between center of salient region and each of the four intersection points.
- hpvb: Horizontal physical visual balance. Split image horizontally. Horizontal physical symmetry (mirroring).
- vpvb: Vertical physical visual balance. Split image vertically. Vertical physical symmetry (mirroring)
- hcvb: Horizontal color visual balance. Split image horizontally. Horizontal mirrored Euclidean cross-pixels distance.
- vcvb: Vertical color visual balance. Split image vertically. Vertical mirrored Euclidean cross-pixels distance.

For simplicity, we will use the validation set for training and test set for testing.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
from collections import Counter
import re
import requests, zipfile, io
import time

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer


In [ ]:
val_df = pd.read_csv('/content/drive/My Drive/foodreviews_val.csv')
test_df = pd.read_csv('/content/drive/My Drive/foodreviews_test.csv')

zip_file_path = '/content/drive/My Drive/images.zip'
extract_dir = '/content/drive/My Drive/'
img_dir = os.path.join(extract_dir, 'images')


In [ ]:
# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Extracted images to {img_dir}")

Extracted images to /content/drive/My Drive/images


## 1. Detecting AI-generated fake reviews using images only

Convolutional neural network (CNN) model with the same model architecture as LeNet-5 from scratch to detecting AI-generated fake reviews from images.


In [ ]:
print("Loading images and defining model...")

# --- 1. Dataset class for loading images ---
class ReviewImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]['ID']
        label = self.df.iloc[idx]['label']
        # Construct the image path by joining the base image directory and the image filename
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        try:
            image = Image.open(img_path).convert('L')  # LeNet-5 uses grayscale
            if self.transform:
                image = self.transform(image)
            return image, label
        except FileNotFoundError:
            print(f"Image file not found: {img_path}")
            # Handle missing images if necessary, for now, raising the error to stop
            raise

# --- 2. LeNet-5 Model ---
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)      # (N,1,32,32) -> (N,6,28,28)
        self.pool1 = nn.AvgPool2d(2, 2)      # (N,6,28,28) -> (N,6,14,14)
        self.conv2 = nn.Conv2d(6, 16, 5)     # (N,6,14,14) -> (N,16,10,10)
        self.pool2 = nn.AvgPool2d(2, 2)      # (N,16,10,10) -> (N,16,5,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)          # 2 classes: authentic/fake

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16*5*5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return x

# --- 3. Data transforms and loaders ---
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

val_dataset = ReviewImageDataset(val_df, img_dir, transform)
test_dataset = ReviewImageDataset(test_df, img_dir, transform)

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)
print("Loading complete!")

Loading images and defining model...
Loading complete!


In [ ]:
print("Training set up...")
# --- Training setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Training model...")
# --- Training loop ---
for epoch in range(10):  # 10 epochs for demonstration
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/10", leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        progress_bar.set_postfix(loss=running_loss / (progress_bar.n + 1))
    print(f"Epoch {epoch+1} finished. Loss: {running_loss / len(val_loader):.4f}")


Training set up...
Training model...


Epoch 1 finished. Loss: 0.6858


Epoch 2 finished. Loss: 0.6093


Epoch 3 finished. Loss: 0.5670


Epoch 4 finished. Loss: 0.5642


...

Epoch 6 finished. Loss: 0.5383


Epoch 7 finished. Loss: 0.5322


Epoch 8 finished. Loss: 0.5278


Epoch 9 finished. Loss: 0.5257


Epoch 10 finished. Loss: 0.5295


In [ ]:
print("Evaluating model...")
# --- Evaluation function ---
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    for images, labels in tqdm(loader, desc="Evaluating", leave=False):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return correct / total

val_acc = evaluate(model, val_loader)
test_acc = evaluate(model, test_loader)

# --- Model summary and results ---
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(model)
print(f"Number of trainable parameters: {count_parameters(model)}")
print(f"Validation accuracy: {val_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

Evaluating model...


LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)
Number of trainable parameters: 61026
Validation accuracy: 0.7520
Test accuracy: 0.7424


## 2. Transfer Learning for AI-generated fake review detection

Used the pretrained [VGG-16](https://pytorch.org/vision/main/models/generated/torchvision.models.vgg16.html) on ImageNet to perform transfer learning in two steps: 1)zero-shot classification; 2)supervised fine-tune (SFT) with the validation set.

In [ ]:
# Load pretrained VGG-16 and modify classifier for 2 classes
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.parameters():
    param.requires_grad = False  # Freeze all layers

# Replace classifier for binary classification
vgg16.classifier[6] = nn.Linear(4096, 2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vgg16 = vgg16.to(device)

# Data transforms (VGG expects 224x224 RGB)
vgg_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x),  # convert grayscale to RGB
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_dataset_vgg = ReviewImageDataset(val_df, img_dir, vgg_transform)
test_dataset_vgg = ReviewImageDataset(test_df, img_dir, vgg_transform)
val_loader_vgg = DataLoader(val_dataset_vgg, batch_size=64, shuffle=True, num_workers=2)
test_loader_vgg = DataLoader(test_dataset_vgg, batch_size=64, shuffle=False, num_workers=2)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:30<00:00, 18.2MB/s]


#### Zero-shot classification (no fine-tuning)

In [ ]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

zero_shot_val_acc = evaluate(vgg16, val_loader_vgg)
zero_shot_test_acc = evaluate(vgg16, test_loader_vgg)

#### Supervised Fine-Tuning (STF) on validation set

In [ ]:
for param in vgg16.classifier[6].parameters():
    param.requires_grad = True  # Only train the last layer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier[6].parameters(), lr=0.001)

for epoch in range(5):  # 5 epochs for demonstration
    vgg16.train()
    running_loss = 0.0
    for images, labels in tqdm(val_loader_vgg, desc=f"Epoch {epoch+1}/5", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(val_loader_vgg)
    print(f"Epoch {epoch+1} finished. Loss: {avg_loss:.4f}")

sft_val_acc = evaluate(vgg16, val_loader_vgg)
sft_test_acc = evaluate(vgg16, test_loader_vgg)


Epoch 1 finished. Loss: 0.5441


Epoch 2 finished. Loss: 0.4481


Epoch 3 finished. Loss: 0.4296


Epoch 4 finished. Loss: 0.4062


Epoch 5 finished. Loss: 0.4106


#### Results comparison

In [ ]:
# Count trainable parameters
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Print results
print("VGG-16 with custom classifier:")
print(vgg16)
print(f"Number of trainable parameters: {count_trainable_params(vgg16)}")
print(f"Zero-shot validation accuracy: {zero_shot_val_acc:.4f}")
print(f"Zero-shot test accuracy: {zero_shot_test_acc:.4f}")
print(f"SFT validation accuracy: {sft_val_acc:.4f}")
print(f"SFT test accuracy: {sft_test_acc:.4f}")


VGG-16 with custom classifier:
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, 

Transfer learning provides a performance improvement when moving from the initial zero-shot stage to the supervised fine-tuning (SFT) stage. In the zero-shot setting, the pre-trained VGG-16 model, without any task-specific training, achieves a baseline performance based on its learned representations from ImageNet. During SFT, the model's classification layers are trained on the specific dataset for AI-generated fake review detection. This adaptation allows the model to learn task-relevant features, leading to a significant boost in both validation and test accuracies compared to the zero-shot results. Thus, transfer learning, specifically the fine-tuning aspect, is crucial for achieving substantial performance gains in this scenario.

## 3. Detecting AI-generated fake reviews using text only

Detector using the customer review text as the source, using LSTM arhitectutue. This LSTM only has one layer, but we will perform two approaches: 1) the embedding of the word may be joint learned with the labels; 2) a transfer learning approach to adopt the pre-trained word embedding models to fill in embedding layers. Used vector wiki-news-300d-1M.vec from [Fasttext](https://fasttext.cc/docs/en/english-vectors.html).

In [ ]:
# Tokenization and Vocabulary
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

# Build vocab from val_df
texts = val_df['text'].tolist()
labels = val_df['label'].tolist()
tokenized_texts = [tokenize(t) for t in texts]
word_counts = Counter([w for sent in tokenized_texts for w in sent])
vocab = ['<PAD>', '<UNK>'] + [w for w, c in word_counts.items() if c >= 2]
word2idx = {w: i for i, w in enumerate(vocab)}

def encode(text, maxlen=100):
    tokens = tokenize(text)
    idxs = [word2idx.get(w, word2idx['<UNK>']) for w in tokens[:maxlen]]
    idxs += [word2idx['<PAD>']] * (maxlen - len(idxs))
    return idxs

maxlen = 100
X = np.array([encode(t, maxlen) for t in val_df['text']])
y = np.array(val_df['label'])

# Split for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Dataset class
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = TextDataset(X_train, y_train)
val_ds = TextDataset(X_val, y_val)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)

# LSTM Model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, pretrained_emb=None, freeze_emb=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        if pretrained_emb is not None:
            self.embedding.weight.data.copy_(torch.from_numpy(pretrained_emb))
            self.embedding.weight.requires_grad = not freeze_emb
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 2)
    def forward(self, x):
        x = self.embedding(x)
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

# Training/Eval function
def train_eval(model, train_loader, val_loader, epochs=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
    # Evaluate
    model.eval()
    all_preds, all_true = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            preds = model(xb).argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_true.extend(yb.numpy())
    acc = accuracy_score(all_true, all_preds)
    return acc

#### Approach 1: Jointly learned embedding

In [ ]:
model1 = LSTMClassifier(len(vocab), 100, 64)
acc1 = train_eval(model1, train_loader, val_loader)
print(f"Accuracy with joint-learned embedding: {acc1:.4f}")

Accuracy with joint-learned embedding: 0.9280


#### Approach 2: Pretrained FastText embedding

In [ ]:
def load_fasttext_embeddings(path, vocab, emb_dim=300):
    embeddings = np.random.normal(scale=0.6, size=(len(vocab), emb_dim)).astype(np.float32)
    found = 0
    with open(path, encoding='utf-8') as f:
        next(f)  # skip header
        for line in tqdm(f, desc="Loading FastText"):
            parts = line.rstrip().split(' ')
            word = parts[0]
            if word in word2idx:
                idx = word2idx[word]
                vec = np.array(parts[1:], dtype=np.float32)
                embeddings[idx] = vec
                found += 1
    print(f"Loaded {found} pretrained vectors.")
    return embeddings

# Download and extract wiki-news-300d-1M.vec
ft_path = 'wiki-news-300d-1M.vec'
if not os.path.exists(ft_path):
    url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip'
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    print("Extracted FastText vectors.")

ft_emb = load_fasttext_embeddings(ft_path, vocab, 300)
model2 = LSTMClassifier(len(vocab), 300, 64, pretrained_emb=ft_emb, freeze_emb=False)
acc2 = train_eval(model2, train_loader, val_loader)
print(f"Accuracy with pretrained FastText embedding: {acc2:.4f}")

Extracted FastText vectors.


Loading FastText: 999994it [00:20, 48782.16it/s]


Loaded 10635 pretrained vectors.
Accuracy with pretrained FastText embedding: 0.9839


The results show that the LSTM model utilizing pre-trained FastText embeddings (accuracy: 0.9839) outperformed the model with jointly learned embeddings (accuracy: 0.9280). The benefit of using pre-trained embeddings lies in their ability to capture semantic relationships between words learned from a massive general-purpose corpus (like Wikipedia and news data). This allows the model to have a richer initial understanding of the language, enabling it to generalize better and achieve higher accuracy, especially when the task-specific training data might be limited. The pre-trained embeddings provide a strong foundation of linguistic knowledge, which the model can then adapt for the specific nuances of fake review detection, leading to improved performance compared to learning embeddings from scratch.

## 4. Detecting AI-generated fake reviews using transformer

Two approaches:
1. Pretrained [RoBERTa model](https://huggingface.co/FacebookAI/roberta-base) to 1) perform zero-shot classification on test set and 2) fine-tune with validation set and predict on test set.

2. Pretrained  [ViT model](https://huggingface.co/google/vit-base-patch16-224) to do the same procedures for zeero-shot classification and SFT technique on images.


###1. RoBERTa for text classification

In [ ]:
# Prepare test set
test_texts = test_df['text'].tolist()
test_labels = test_df['label'].tolist()
val_texts = val_df['text'].tolist()
val_labels = val_df['label'].tolist()

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

class ReviewTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

test_dataset = ReviewTextDataset(test_texts, test_labels, tokenizer)
val_dataset = ReviewTextDataset(val_texts, val_labels, tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

#### 1.1. Zero-shot: RoBERTa as feature extractor (no fine-tuning)

In [ ]:
roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
roberta.eval()
def roberta_predict(model, dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size=32)
    preds = []
    for batch in loader:
        with torch.no_grad():
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
            pred = outputs.logits.argmax(dim=1).cpu().numpy()
            preds.extend(pred)
    return preds

zero_shot_preds = roberta_predict(roberta, test_dataset)
zero_shot_acc = accuracy_score(test_labels, zero_shot_preds)
print(f"RoBERTa zero-shot test accuracy: {zero_shot_acc:.4f}")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)eb5afc8b2b397fe5e04beabb9b1ef355255ade81:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTa zero-shot test accuracy: 0.5072


####  1.2 Fine-tune RoBERTa on validation set, test on test set

In [ ]:
training_args = TrainingArguments(
    output_dir='./results', num_train_epochs=2, per_device_train_batch_size=16,
    per_device_eval_batch_size=32, eval_strategy="no", logging_steps=100, save_strategy="no", # Changed evaluation_strategy to eval_strategy
    learning_rate=2e-5, weight_decay=0.01, disable_tqdm=True
)
roberta_ft = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
trainer = Trainer(
    model=roberta_ft,
    args=training_args,
    train_dataset=val_dataset,
)
trainer.train()
ft_preds = roberta_predict(roberta_ft, test_dataset)
ft_acc = accuracy_score(test_labels, ft_preds)
print(f"RoBERTa fine-tuned test accuracy: {ft_acc:.4f}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: julianamafini (julianamafini-nova-sbe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'loss': 0.1879, 'grad_norm': 0.03574594855308533, 'learning_rate': 1.6071428571428572e-05, 'epoch': 0.3968253968253968}
{'loss': 0.0148, 'grad_norm': 0.009927497245371342, 'learning_rate': 1.2103174603174603e-05, 'epoch': 0.7936507936507936}
{'loss': 0.0194, 'grad_norm': 0.03694852069020271, 'learning_rate': 8.134920634920636e-06, 'epoch': 1.1904761904761905}
{'loss': 0.0016, 'grad_norm': 0.004436303395777941, 'learning_rate': 4.166666666666667e-06, 'epoch': 1.5873015873015874}
{'loss': 0.0002, 'grad_norm': 0.004167582839727402, 'learning_rate': 1.9841269841269841e-07, 'epoch': 1.9841269841269842}
{'train_runtime': 10412.6063, 'train_samples_per_second': 0.774, 'train_steps_per_second': 0.048, 'train_loss': 0.044445347806368586, 'epoch': 2.0}
RoBERTa fine-tuned test accuracy: 0.9983


The results show that using a transformer model like RoBERTa significantly improves performance when fine-tuned on the specific task. The zero-shot classification achieved an accuracy of only 0.5072, barely above random chance, indicating that the pre-trained model alone cannot generalize well to fake review detection without adaptation. However, after fine-tuning on the validation set, the test accuracy jumped to 0.9983, demonstrating that the transformer architecture is highly effective at capturing task-specific patterns when given labeled data. That said, such a near-perfect accuracy may also indicate potential overfitting, especially if the test set is small or not representative, so further validation on unseen data would be important to confirm the model's generalization ability.

### 2. ViT for image classification

In [ ]:
# Prepare image dataset for ViT
vit_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
vit_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x)
])

class ReviewImageDatasetViT(Dataset):
    def __init__(self, df, img_dir, extractor):
        self.df = df
        self.img_dir = img_dir
        self.extractor = extractor
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]['ID']
        label = self.df.iloc[idx]['label']
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        image = Image.open(img_path).convert('RGB')
        inputs = self.extractor(images=image, return_tensors="pt")
        item = {k: v.squeeze(0) for k, v in inputs.items()}
        item['labels'] = torch.tensor(label)
        return item

test_img_dataset = ReviewImageDatasetViT(test_df, img_dir, vit_extractor)
val_img_dataset = ReviewImageDatasetViT(val_df, img_dir, vit_extractor)

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


#### 2.1 Zero-shot ViT

In [ ]:
vit = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=2, ignore_mismatched_sizes=True)
vit.eval()
def vit_predict(model, dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size=16)
    preds = []
    for batch in tqdm(loader, desc="Predicting with ViT"):
        with torch.no_grad():
            outputs = model(pixel_values=batch['pixel_values'])
            pred = outputs.logits.argmax(dim=1).cpu().numpy()
            preds.extend(pred)
    return preds

vit_zero_preds = vit_predict(vit, test_img_dataset)
vit_zero_acc = accuracy_score(test_df['label'], vit_zero_preds)
print(f"ViT zero-shot test accuracy: {vit_zero_acc:.4f}")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Predicting with ViT: 100%|██████████| 252/252 [46:04<00:00, 10.97s/it]

ViT zero-shot test accuracy: 0.5303


#### 2.2 Fine-tune ViT on validation set, test on test set

In [ ]:
vit_ft = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=2, ignore_mismatched_sizes=True)
training_args_img = TrainingArguments(
    output_dir='./results_vit', num_train_epochs=1, per_device_train_batch_size=16,
    per_device_eval_batch_size=16, eval_strategy="no", logging_steps=100, save_strategy="no",
    learning_rate=2e-5, weight_decay=0.01, disable_tqdm=False
)
trainer_img = Trainer(
    model=vit_ft,
    args=training_args_img,
    train_dataset=val_img_dataset,
)
trainer_img.train()
vit_ft_preds = vit_predict(vit_ft, test_img_dataset)
vit_ft_acc = accuracy_score(test_df['label'], vit_ft_preds)
print(f"ViT fine-tuned test accuracy: {vit_ft_acc:.4f}")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: julianamafini (julianamafini-nova-sbe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,0.271700
200,0.033200


Predicting with ViT: 100%|██████████| 252/252 [32:32<00:00,  7.75s/it]

ViT fine-tuned test accuracy: 0.9901


The ViT (Vision Transformer) model also demonstrated a significant improvement in performance after fine-tuning. In the zero-shot setting, it achieved a test accuracy of 0.5303, indicating limited capability to generalize to the image classification task without task-specific training. However, after applying supervised fine-tuning (SFT), the test accuracy rose to 0.9901. This substantial gain highlights the effectiveness of fine-tuning transformer-based models on domain-specific data. Similar to the RoBERTa case, such a high accuracy may suggest potential overfitting, especially if the dataset is small or lacks diversity, so additional evaluation on an external or more diverse test set would be important to confirm generalizability.

## Detecting AI-generated fake reviews with LLM model

LLM model to perform review detection text using Google Gemini API.


In [ ]:
# Set your Gemini API key here
GEMINI_API_KEY = "XX"
GEMINI_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key=" + GEMINI_API_KEY

def gemini_classify(review, examples=None):
    if examples:
        # Few-shot: prepend more diverse and balanced examples to the prompt
        prompt = "You are a fake review detector. Label each review as 0 (authentic) or 1 (AI-generated).\n"
        for ex in examples:
            # Truncate example text for prompt length
            ex_text = ex['text'][:200].replace('\n', ' ')
            prompt += f"Review: {ex_text}\nLabel: {ex['label']}\n"
        review_text = review[:200].replace('\n', ' ')
        prompt += f"Review: {review_text}\nLabel:"
    else:
        # Zero-shot: just ask for the label
        review_text = review[:200].replace('\n', ' ')
        prompt = f"Is the following review authentic (0) or AI-generated (1)?\nReview: {review_text}\nLabel:"

    data = {
        "contents": [{"parts": [{"text": prompt}]}],
        "generationConfig": {"maxOutputTokens": 10}
    }
    response = requests.post(GEMINI_URL, json=data)
    try:
        label = response.json()['candidates'][0]['content']['parts'][0]['text'].strip()
        # rint(label)
        # Extract first digit (0 or 1)
        label = int(label[0]) if label[0] in "01" else 0
    except Exception:
        label = 0
    return label

# Evaluate on a small subset for demo
N = 20  # Number of samples to test
sample_df = test_df.sample(N, random_state=42)
texts = sample_df['text'].tolist()
true_labels = sample_df['label'].tolist()

# Zero-shot
zero_shot_preds = []
for t in texts:
    zero_shot_preds.append(gemini_classify(t))
    time.sleep(1)  # avoid rate limit

zero_shot_acc = accuracy_score(true_labels, zero_shot_preds)
print(f"Gemini zero-shot accuracy (N={N}): {zero_shot_acc:.4f}")

# Few-shot (use 20 diverse examples: 10 authentic, 10 AI-generated)
auth_examples = val_df[val_df['label'] == 0].sample(10, random_state=1)[['text', 'label']].to_dict('records')
ai_examples = val_df[val_df['label'] == 1].sample(10, random_state=2)[['text', 'label']].to_dict('records')
few_shot_examples = auth_examples + ai_examples

few_shot_preds = []
for t in texts:
    few_shot_preds.append(gemini_classify(t, examples=few_shot_examples))
    time.sleep(1)

few_shot_acc = accuracy_score(true_labels, few_shot_preds)
print(f"Gemini few-shot accuracy (N={N}): {few_shot_acc:.4f}")

Gemini zero-shot accuracy (N=20): 0.4500
Gemini few-shot accuracy (N=20): 0.5500


In the zero-shot setting, the model achieved an accuracy of 0.4500, indicating limited ability to detect fake reviews without prior context. However, when provided with a few labeled examples in the prompt (few-shot), the accuracy increased to 0.5500. Although the improvement is modest, it suggests that the model benefits from seeing examples of the task, helping it better understand the labeling criteria. The small test size (N=20) and the lightweight nature of Gemini Flash may limit the observed gains, but the results still support the general trend that few-shot prompting improves performance over zero-shot for this task.